### WLLS_II_estimator.ipynb
- Siep Dokter
- Emil Jousimaa
- Oleksandr Sosovskyy
- Mario Gabriele Carofano

> This file contains the implementation of an alternative WLLS estimator, named WLLS_II, as requested in the 4th task, used for estimate the Target Coordinates starting from the RSS information coming from the anchors.

> In addition, at the end of file, there are also plots showing the actual position of the target and the anchors, and the estimated position of the target obtained from the execution of the WLS estimator.

In [ ]:
# IMPORTS
import import_ipynb
import constants
import auxfunc
import pandas as pd
import numpy as np
import pprint
import math

In [ ]:
def calculate_AII_matrix(anchor_coords, r):
	'''
	Calculates the 'AII' matrix of the system of equations [R2, (13)] which the WLLS-II method solves.
	This is the implementation of [R2, (14)].

	Parameters:
	anchor_coords (list) : A list containing one list for each anchor, e.g. the 2D-coordinates of each anchor.
	r (int) : An integer representing the index in 'anchor_coords' of the reference anchor.

	Returns:
	Returns a 2D numpy.ndarray which values
	are elements of the 'AII' matrix for the selected scenario.
	'''

	n_anchors = len(anchor_coords)
	AII = np.zeros((n_anchors, 2))

	for i in range(n_anchors):
		AII[i, 0] = anchor_coords[i][0] - anchor_coords[r][0]
		AII[i, 1] = anchor_coords[i][1] - anchor_coords[r][1]
	
	return 2 * AII

In [ ]:
def calculate_bII_vector(anchor_coords, estimated_distances, r):
	'''
	Calculates the 'b' vector of the system of equations which the WLS method solves.
	This is the implementation of (5) on the reference paper.

	Parameters:
	anchor_coords (list) : A list containing one list for each anchor, e.g. the 2D-coordinates of each anchor.
	estimated_distances (list):
	A list containing one real number for each anchor,
	e.g. the distance estimation between each anchor and the target.
	r (int) : An integer representing the index in 'anchor_coords' of the reference anchor.

	Returns:
	Returns a 1D numpy.ndarray which values
	are elements of the 'bII' vector for the selected configuration.
	'''

	n_anchors = len(anchor_coords)
	first_term = math.pow(estimated_distances[r], 2)
	third_term = math.pow(anchor_coords[r][0], 2) + math.pow(anchor_coords[r][1], 2)
	bII = []

	for i in range(n_anchors):
		second_term = math.pow(estimated_distances[i], 2)
		fourth_term = third_term = math.pow(anchor_coords[i][0], 2) + math.pow(anchor_coords[i][1], 2)
		bII.append(first_term - second_term - third_term + fourth_term)
	
	return np.array(bII)


In [ ]:
def calculate_CII_matrix(estimated_distances, r):
    """
	Calculates, for the selected configuration, the 'CII' diagonal matrix.
	This is the implementation of [R2-13, sect. IV].

	Parameters:
	estimated_distances (list):
	A list containing one real number for each anchor,
	e.g. the distance estimation between each anchor and the target.
	r (int) : An integer representing the index in 'anchor_coords' of the reference anchor.

	Returns:
    Returns a 2D numpy.ndarray which values are elements
    of the 'CII' covariance diagonal matrix for the selected configuration.
	"""

    n_anchors = len(estimated_distances)

    first_term = 4 * math.pow(estimated_distances[r], 2) * math.pow(constants.STANDARD_DEVIATION, 2)
    second_term = 2 * math.pow(constants.STANDARD_DEVIATION, 4)

    covariance = []

    for i in range(n_anchors):
        third_term = 4 * math.pow(estimated_distances[i], 2) * math.pow(constants.STANDARD_DEVIATION, 2)
        covariance.append(first_term + second_term + third_term + second_term)
    
    return np.diag(covariance)


In [ ]:
def calculate_WLLS_II_output(AII, bII, CII):
	"""
	Calculates the Weighted Linear Least Squares (WLLS) position estimate.
	This is the implementation of (6) on the reference paper.

	Parameters:
	A (numpy.ndarray): Matrix A.
	W (numpy.ndarray): Weight matrix W.
	b (numpy.ndarray): Vector b.

	Returns:
	position (numpy.ndarray): WLS position estimate.
	"""
	
	AII_transpose = np.transpose(AII)
	CII_inverse = np.linalg.inv(CII)
    
	mult = np.matmul(AII_transpose, CII_inverse)
    
	first_term = np.linalg.inv(np.matmul(mult, AII))
	second_term = np.matmul(mult, bII)
    
	return np.matmul(first_term, second_term)

In [ ]:
def apply_WLLS_II_estimator(directory_path):
	'''
	Applies the WLLS-II estimator, as shown in [R2].
	It is used for estimate the target's position from the anchors' position.

	Parameters:
	directory_path (str):
	The path of the directory which contains the dataset.
	If the directory does not contain any dataset, then the execution navigate into
	'constants.BLT_DIRECTORY' and 'constants.WIFI_DIRECTORY' in order to get datasets
	from both directories.

	Returns:
	data (dict):
	It is a dictionary with 'Actual' and 'Estimated' keys.
	Contains all the salient information retrieved from the reading of the dataset
	and from the application of the WLLS-II estimator, respectively.
	'''
	
	dataframes = []
	data = {}; data["Actual"] = {}; data["Estimated"] = {}
	
	anchors = auxfunc.get_anchors_name(directory_path)
	n_anchors = len(anchors)

	if n_anchors == 0:
		blt_anchors = auxfunc.get_anchors_name(directory_path + constants.BLT_DIRECTORY)
		wifi_anchors = auxfunc.get_anchors_name(directory_path + constants.WIFI_DIRECTORY)
		n_anchors = len(blt_anchors) + len(wifi_anchors)

		# Concatenates the blt and wifi estimations
		for b in blt_anchors:
			dataframes.append(pd.read_csv(
				directory_path + constants.BLT_DIRECTORY + b,
				sep=constants.CSV_FIELDS_SEPARATOR
			))
		for w in wifi_anchors:
			dataframes.append(pd.read_csv(
				directory_path + constants.WIFI_DIRECTORY + w,
				sep=constants.CSV_FIELDS_SEPARATOR
			))
	else:
		for a in anchors:
			dataframes.append(pd.read_csv(
				directory_path + a,
				sep=constants.CSV_FIELDS_SEPARATOR
			))

	length = auxfunc.calculate_smallest_dataset(dataframes)
	burst_quantity = auxfunc.calculate_burst_quantity(length)

	data["Actual"]["Distance Target - Anchor"] = []
	data["Actual"]["Anchor Coordinates"] = []
	for a in range(n_anchors):
		data["Actual"]["Distance Target - Anchor"].append(dataframes[a]["Distance Target - Anchor [m]"][0])
		data["Actual"]["Target Coordinates"] = [eval(i) for i in dataframes[a]["Target Coordinates [m]"][0].split(", ")]
		data["Actual"]["Anchor Coordinates"].append([eval(i) for i in dataframes[a]["Relative Coordinates [m]"][0].split(", ")])

	for c in range(burst_quantity):
		data["Estimated"][c] = {}
		data["Estimated"][c]["Average RSS"] = []
		data["Estimated"][c]["Distance Target - Anchor"] = []
		
		for a in range(n_anchors):
			average_RSS = auxfunc.calculate_average_RSS(dataframes[a], length)
			estimated_distances = auxfunc.calculate_target_anchors_estimation(average_RSS)
			data["Estimated"][c]["Average RSS"].append(average_RSS[c])
			data["Estimated"][c]["Distance Target - Anchor"].append(estimated_distances[c])

		AII = calculate_AII_matrix(data["Actual"]["Anchor Coordinates"], 0)
		bII = calculate_bII_vector(data["Actual"]["Anchor Coordinates"], data["Estimated"][c]["Distance Target - Anchor"], 0)
		CII = calculate_CII_matrix(data["Estimated"][c]["Distance Target - Anchor"], 0)
		
		data["Estimated"][c]["A matrix"] = AII
		data["Estimated"][c]["b vector"] = bII
		data["Estimated"][c]["W matrix"] = CII
		data["Estimated"][c]["Target Coordinates"] = calculate_WLLS_II_output(AII, bII, CII)[[0,1]]

	return data


In [ ]:
data = {}

data["Scenario A"] = {}
data["Scenario B"] = {}
data["Scenario C"] = {}

data["Scenario A"]["RPI-WiFi"] = apply_WLLS_II_estimator(constants.SCENARIO_A_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario A"]["RPI-BLT"] = apply_WLLS_II_estimator(constants.SCENARIO_A_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario A"]["RPI-Hybrid"] = apply_WLLS_II_estimator(constants.SCENARIO_A_PATH)

data["Scenario B"]["RPI-WiFi"] = apply_WLLS_II_estimator(constants.SCENARIO_B_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario B"]["RPI-BLT"] = apply_WLLS_II_estimator(constants.SCENARIO_B_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario B"]["RPI-Hybrid"] = apply_WLLS_II_estimator(constants.SCENARIO_B_PATH)

data["Scenario C"]["RPI-WiFi"] = apply_WLLS_II_estimator(constants.SCENARIO_C_PATH + "RPI/RSS_WiFi_Dataset/")
data["Scenario C"]["RPI-BLT"] = apply_WLLS_II_estimator(constants.SCENARIO_C_PATH + "RPI/RSS_BLT_Dataset/")
data["Scenario C"]["RPI-Hybrid"] = apply_WLLS_II_estimator(constants.SCENARIO_C_PATH)

# np.set_printoptions(suppress=True)
# pprint.pprint(data)
# for configuration in data["Scenario A"]["RPI-WiFi"]["Estimated"].values():
#     pprint.pprint(configuration["Target Coordinates"])

In [ ]:
print("Scenario A:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario A"]["RPI-Hybrid"]), "\n")

print("Scenario B:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario B"]["RPI-Hybrid"]), "\n")

print("Scenario C:")
print("𝜭W:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-WiFi"]))
print("𝜭B:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-BLT"]))
print("𝜭H:", auxfunc.calculate_rmse(data["Scenario C"]["RPI-Hybrid"]))

In [ ]:
auxfunc.plot_data("WLLS-II", "Scenario A", data["Scenario A"])
auxfunc.plot_data("WLLS-II", "Scenario B", data["Scenario B"])
auxfunc.plot_data("WLLS-II", "Scenario C", data["Scenario C"])